# Data Preparation Code
## Prepare data from directories

Importing all need libraries

In [1]:
from __future__ import print_function, division
from skimage import io, transform
from random import randint

import os, uuid, glob, warnings
import numpy as np

warnings.filterwarnings("ignore")

Pathes for images and their annotations

In [21]:
ORIGINAL_DATA_DIR = '../data/fl27/original'
CROPPED_DATA_DIR = '../data/fl27/images'
ORIGINAL_ANNOTATION = '../data/fl27/annotation.txt'
CROPPED_ANNOTATION = '../data/fl27/crop_annotation.txt'

TRAIN_SET = '../annotations/trainset.txt'
TEST_SET = '../annotations/testset.txt'

Original images of logos

In [3]:
original_files = glob.glob(os.path.join(ORIGINAL_DATA_DIR, '*.jpg'))

Reading from txt files

In [4]:
def read_from_annotation(path):
    file = open(path, "r")
    content = file.readlines()
    new = [x.split(" ")[:-1] for x in content]
    return new

Preparing labels list from annotation

In [25]:
def prepare_labels(annotaton_path):
    arr = read_from_annotation(annotaton_path)
    labels = []
    for item in (arr):
        l = item[1]
        if l not in labels:
            labels.append(l)
        else:
            continue
    return labels
LABELS = prepare_labels(ORIGINAL_ANNOTATION)

Creating crops and resize them for creating test and train dataset

In [7]:
def create_resized_crops(input_annotation, input_directory,
                        output_annotation, output_directory):
    annotation = read_from_annotation(input_annotation)
    output_annotation = open(output_annotation, 'w')
    for i, data in zip(range(len(annotation)),annotation):
        img_name = data[0]
        img_path = os.path.join(input_directory, img_name)
        image = io.imread(img_path)
        positions = data[-4:]
        x1 = int(positions[0])
        y1 = int(positions[1])
        x2 = int(positions[2])
        y2 = int(positions[3])

        if x1 > x2:
            tmp = x1
            x1 = x2
            x2 = tmp

        if y1 > y2:
            tmp = y1
            y1 = y2
            y2 = tmp
        
        
            
        target = data[1]
        crop_resizer(image, target, x1,x2,y1,y2,output_annotation, output_directory)
        crop_resizer(image, target, x1-40,x2,y1,y2,output_annotation, output_directory)
        crop_resizer(image, target, x1,x2+40,y1,y2,output_annotation, output_directory)
        crop_resizer(image, target, x1,x2,y1-40,y2,output_annotation, output_directory)
        crop_resizer(image, target, x1,x2,y1,y2+40,output_annotation, output_directory)

def crop_resizer(image,target, x1,x2,y1,y2, writer,directory):
    crop_img = image[y1:y2, x1:x2]
    file_name = str(uuid.uuid4().hex)+'.jpg'
    try:
        try:
            resized = transform.resize(crop_img,(224, 224))
            io.imsave(os.path.join(directory, file_name), resized)
            note = "{} {} \n".format(file_name, target)
            writer.write(note)
        except ValueError:
            pass
    except IndexError:
        pass
#create_resized_crops(ORIGINAL_ANNOTATION, ORIGINAL_DATA_DIR, CROPPED_ANNOTATION, CROPPED_DATA_DIR)

Dividing images into train and test dataset, prepare SET.TXT files

In [22]:
def train_test_split(crop_ann_path, out_directory, test_size = 1000):
    train = read_from_annotation(crop_ann_path)
    test = []
    for i in range(test_size):
        randIndex = randint(0,len(train)-1)
        test.append(train[randIndex])
        del(train[randIndex])

    trainset = file(TRAIN_SET, "w")
    testset = file(TEST_SET, "w")

    for item in train:
        tmp = "{} {} \n".format(item[0], item[1])
        trainset.write(tmp)
    trainset.close()

    for item in test:
        tmp = "{} {} \n".format(item[0], item[1])
        testset.write(tmp)
    testset.close()
train_test_split(CROPPED_ANNOTATION,RESIZED_ANNOTATION,5000)

## Creating pytorch dataset 

Importing libraries for pytorch

In [12]:
import torch
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
from torch import optim
plt.ion()

Main pytorch dataset class

In [13]:
class MyDataset(Dataset):
    def __init__(self, txt_file, root):
        self.txt_file = txt_file
        self.root = root

    def __len__(self):
        return self.txt_file.shape[0]

    def __getitem__(self, id):
        img_name = os.path.join(self.root, self.txt_file[id][0])
        img = io.imread(img_name)
        logo = int(self.txt_file[id][1])
        image = img.transpose((2, 0, 1))
        img = torch.FloatTensor(image)
        return img,logo

My main labels(in future I want to add new class)

In [27]:
print(LABELS)

['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex', 'Ferrari', 'Ford', 'Google', 'Heineken', 'HP', 'Intel', 'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma', 'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone', 'Yahoo']


Convert function from simple list into numpy array with categorization

In [30]:
def prepare_num_dataset(annotation_path, set_path):
    arr = read_from_annotation(set_path)
    out = []
    for item in arr:
        tmp = [item[0], LABELS.index(item[1])]
        out.append(tmp)
    out = np.array(out)
    return out

Creating main train and test set, which are suitable for pytorch

In [37]:
train_data = prepare_num_dataset(CROPPED_ANNOTATION, TRAIN_SET)
test_data = prepare_num_dataset(CROPPED_ANNOTATION, TEST_SET)
trainset = MyDataset(train_data, CROPPED_DATA_DIR)
testset = MyDataset(test_data, CROPPED_DATA_DIR)
print("Train size {} items and test size {} items".format(len(trainset), len(testset)))

Train size 16185 items and test size 5000 items


Creating Data Loaders, which send for training batches with fixed size

In [39]:
batch_size = 40
train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                            batch_size=batch_size,
                                            shuffle=False)

# Training Convolutional Neural Network with 27 classes

## Without any class balancing

Initialize hyper parameters of CNN and number of classes

In [40]:
num_epochs = 1
learning_rate = 0.001
momentum = 0.9

n_classes = 27

Creating CNN class and initialize it with number of classes

In [43]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, n_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
    
cnn = CNN(n_classes)

Training convolutional neural network

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr = learning_rate, momentum=momentum)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                %(epoch+1, num_epochs, i+1, len(trainset)//batch_size, loss.data[0]))

# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn-vol-2.pt')

### Testing neural network on validation set

In [ ]:
cnn = CNN(n_classes)
cnn.load_state_dict(torch.load('cnn-vol-2.pt'))
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))